In [1]:
import pandas as pd
df = pd.read_fwf(r'C:\Users\D Deepak Prasanna\Desktop\class\Internship\Major Project\Reviews.txt',header=None,index=False,names=['Rating', 'Review'])

In [2]:
df.head()

,Rating,Review
0,__label__2,Great CD: My lovely Pat has one of the GREAT v...
1,__label__2,One of the best game music soundtracks - for a...
2,__label__1,Batteries died within a year ...: I bought thi...
3,__label__2,"works fine, but Maha Energy is better: Check o..."
4,__label__2,Great for the non-audiophile: Reviewed quite a...


In [3]:
df['Rating'] = df['Rating'].replace(['__label__1'],'Negative')
df['Rating'] = df['Rating'].replace(['__label__2'],'Positive')

In [4]:
df.head()

,Rating,Review
0,Positive,Great CD: My lovely Pat has one of the GREAT v...
1,Positive,One of the best game music soundtracks - for a...
2,Negative,Batteries died within a year ...: I bought thi...
3,Positive,"works fine, but Maha Energy is better: Check o..."
4,Positive,Great for the non-audiophile: Reviewed quite a...


In [5]:
import nltk
nltk.download('stopwords')
stopwords_list = nltk.corpus.stopwords.words('english')
stopwords_list.remove('no')
stopwords_list.remove('not')

[nltk_data] Downloading package stopwords to C:\Users\D Deepak
[nltk_data]     Prasanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text = re.sub(pattern,"",text)
  return text

In [7]:
df['Rating'] = df['Rating'].replace(['__label__1'],'Negative')
df['Rating'] = df['Rating'].replace(['__label__2'],'Positive')

In [8]:
df

,Rating,Review
0,Positive,Great CD: My lovely Pat has one of the GREAT v...
1,Positive,One of the best game music soundtracks - for a...
2,Negative,Batteries died within a year ...: I bought thi...
3,Positive,"works fine, but Maha Energy is better: Check o..."
4,Positive,Great for the non-audiophile: Reviewed quite a...
...,...,...
399995,Negative,Unbelievable- In a Bad Way: We bought this Tho...
399996,Negative,"Almost Great, Until it Broke...: My son reciev..."
399997,Negative,Disappointed !!!: I bought this toy for my son...
399998,Positive,Classic Jessica Mitford: This is a compilation...


In [9]:
import re
import string
#remove special characters
def remove_sp(text):
    text = text.lower()
    text = re.sub('\[.*?\]',"",text)
    text = re.sub('[%s]' %re.escape(string.punctuation), "", text)
    text = re.sub('\w*\d\w',"",text)
    text = re.sub('[''""_]', "", text)
    text = re.sub('\n',"", text)
    return text

In [10]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

#remove stopwords
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopwords_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

In [11]:
df.Review = df.Review.apply(lambda x:x.lower())

df.Review = df.Review.apply(remove_sp)

df.Review = df.Review.apply(remove_stopwords)

In [12]:
x = df['Review'].values
y = df['Rating'].values

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state = 0)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

clf = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [16]:
clf.fit(x_train, y_train)
predicted = clf.predict(x_test)

In [19]:
from  sklearn.metrics  import accuracy_score
predicted = clf.predict(x_test)
print(accuracy_score(y_test,predicted))

0.84687


In [20]:
predicted

array(['Negative', 'Positive', 'Negative', ..., 'Negative', 'Positive',
       'Negative'], dtype='<U8')

In [21]:
y_test

array(['Negative', 'Positive', 'Negative', ..., 'Negative', 'Negative',
       'Negative'], dtype=object)

In [22]:
import pickle
pickle.dump(clf,open('SentimentAnalysis.p','wb'))

In [23]:
model = pickle.load(open('SentimentAnalysis.p','rb'))

In [26]:
text = input("Enter Text: ")
text = text.lower()
text = remove_sp(text)
text = remove_stopwords(text)
text = [text]
print(text)
y_out = model.predict(text)
print(f'Predicted output: {y_out}')
CATEGORIES = ['Negative', 'Positive']
q = model.predict_proba(text)
for index, item in enumerate(CATEGORIES):
      print(f'{item} : {q[0][index]*100}%')

Enter Text: Good product liked it alot looking forward many such things in future
['good product liked alot looking forward many things future']
Predicted output: ['Positive']
Negative : 31.792435067934516%
Positive : 68.2075649320656%


In [43]:
%%writefile app.py
import streamlit as st
import os
import matplotlib.pyplot as plt
import numpy as np
import nltk
import re
import string
import pickle
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
import nltk
nltk.download('stopwords')
stopwords_list = nltk.corpus.stopwords.words('english')
stopwords_list.remove('no')
stopwords_list.remove('not')
model = pickle.load(open('SentimentAnalysis.p','rb'))

st.title("Sentiment Analysis")
st.subheader("Enter Text: ")
text = st.text_input(" ")

def remove_sp(text):
        text = text.lower()
        text = re.sub('\[.*?\]',"",text)
        text = re.sub('[%s]' %re.escape(string.punctuation), "", text)
        text = re.sub('\w*\d\w',"",text)
        text = re.sub('[''""_]', "", text)
        text = re.sub('\n',"", text)
        return text
    
    #remove stopwords
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopwords_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

text = text.lower()
text = remove_sp(text)
text = remove_stopwords(text)
st.write(text)
text = [text]
y_out = model.predict(text)

if st.button("Predict"):
    
    st.write(f'PREDICTED OUTPUT: {y_out}')

    if (y_out == "Positive"):
        st.write("\U0001f600")
    else:
        st.write("😞")
    
    CATEGORIES = ['Negative', 'Positive']
    q = model.predict_proba(text)
    for index, item in enumerate(CATEGORIES):
          st.write(f'{item} : {q[0][index]*100}%')
    

Overwriting app.py


In [70]:
%%writefile app.py
import streamlit as st
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import nltk
import re
import string
import pickle
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
import nltk
nltk.download('stopwords')
stopwords_list = nltk.corpus.stopwords.words('english')
stopwords_list.remove('no')
stopwords_list.remove('not')
model = pickle.load(open('SentimentAnalysis.p','rb'))

st.title("Sentiment Analysis")

nav = st.sidebar.radio("Navigation",["Home", "Prediction"])

if nav == "Home":
    image = Image.open(r"C:\Users\D Deepak Prasanna\Desktop\class\Internship\Major Project\image.jfif")
    st.image(image,width = 500)
    st.write("Opinion information is very important for businesses and manufacturers. They often want to know in time what consumers and the public  think of their products and services. However, it is not realistic to manually read every post on the website and extract useful viewpoint information from it. If you do it manually, there is too much data. Sentiment analysis allows large-scale processing of data in an efficient and cost-effective manner. This Project used a dataset of the Amazon reviews and then built a model to predict the sentiment of the comment given the comment declaration by using Python and machine learning algorithm- Naïve Bayes.")
if nav == "Prediction":
    st.subheader("Enter Text: ")
    text = st.text_input(" ")

    def remove_sp(text):
            text = text.lower()
            text = re.sub('\[.*?\]',"",text)
            text = re.sub('[%s]' %re.escape(string.punctuation), "", text)
            text = re.sub('\w*\d\w',"",text)
            text = re.sub('[''""_]', "", text)
            text = re.sub('\n',"", text)
            return text

        #remove stopwords
    def remove_stopwords(text):
      tokens = tokenizer.tokenize(text)
      tokens = [token.strip() for token in tokens]
      filtered_tokens = [token for token in tokens if token not in stopwords_list]
      filtered_text = ' '.join(filtered_tokens)
      return filtered_text

    text = text.lower()
    text = remove_sp(text)
    text = remove_stopwords(text)
    st.write(text)
    text = [text]
    y_out = model.predict(text)

    if st.button("Predict"):

        st.write(f'PREDICTED OUTPUT: {y_out}')

        if (y_out == "Positive"):
            st.write("\U0001f600")
        else:
            st.write("😞")

        CATEGORIES = ['Negative', 'Positive']
        q = model.predict_proba(text)
        for index, item in enumerate(CATEGORIES):
              st.write(f'{item} : {q[0][index]*100}%')

Overwriting app.py
